In [5]:
import numpy as np
import matplotlib.pyplot as plt

from CADMium import Pssolver
from CADMium import Psgrid
from CADMium import Partition
from CADMium import Inverter

%matplotlib widget

a = 5.122/2
#Nuclear charge for fragments A and B
Za, Zb = 3,3
#Set polarization 1-Unpolarized, 2-Polarized
pol = 2
#Fragment a electrons [alpha, beta]
Nmo_a = [[2,1]] #Number of molecular orbitals to calculate
N_a   = [[2,1]]
#Ensemble mix
nu_a = 1
#Fragment b electrons
Nmo_b = [[2,1]]
N_b   = [[2,1]]
#Ensemble mix
nu_b = 1

#Molecular elctron configuration
Nmo_m = [[3,3]]
N_m   = [[3,3]]
 

#Set up grid. Increase to [7,12,12]
NP = 5
NM = [4,4]
L = np.arccosh(15/a)
loc = np.array(range(-4,5)) #Stencil outline

grid = Psgrid(NP, NM, a, L, loc)
grid.initialize()

part = Partition(grid, Za, Zb, pol, Nmo_a, N_a, nu_a, Nmo_b, N_b, nu_b, { "kinetic_part_type" : 'inversion',
                                                                          "ab_sym"            : True,
                                                                          "ens_spin_sym"      : False,})

#Setup inverter object
mol_solver = Pssolver(grid, Nmo_m, N_m)
part.inverter = Inverter(grid, mol_solver, {"invert_type"     : "wuyang",
                                            "tol_invert"      : 1e-10,
                                            "max_iter_invert" : 40,
                                            "disp"            : True,
                                            "ab_sym"          : True,
                                            "ens_spin_sym"    : False,})

part.optPartition.isolated = True
part.scf({"disp"  : True,
          "alpha" : [0.6],
          "e_tol" : 1e-12})

In [6]:
part.KSa.solver[0][0].optSolver.iter_lin_solver = False
part.KSa.solver[0][1].optSolver.iter_lin_solver = False

In [7]:
part.optPartition.isolated   = False

part.scf({"disp"       : True,
          "alpha"      : [0.6],
          "max_iter"   : 50,
          "e_tol"      : 1e-10,
          "continuing" : True})



`xtol` termination condition is satisfied.
`xtol` termination condition is satisfied.


TypeError: 'NoneType' object is not subscriptable

In [ ]:
full, x,y = grid.plotter(part.V.vp[:,0] + part.V.vp[:,1])
x_axis,vp = grid.axis_plot(part.V.vp[:,0] + part.V.vp[:,1])

In [ ]:
fig = plt.figure(dpi=150)
plt.contourf(x,y,full, levels=30, cmap="viridis")
# plt.xlim([-5,5])
# plt.ylim([-5,5])
plt.colorbar()
# plt.show()

In [ ]:
fig = plt.figure(dpi=120)
plt.plot(x_axis,vp)
# plt.xlim(0,5)

In [ ]:
%matplotlib widget

fig, ax = plt.subplots(subplot_kw={"projection": "3d"})
surf = ax.plot_surface(x, y, full, cmap="viridis",
                       linewidth=0, antialiased=True)

ax.grid(False)
ax.set_facecolor("white")
fig.show()